In [2]:
import tensorflow as tf 
from tensorflow.keras.models import load_model
import pickle
import pandas as pd 
import numpy as np

In [3]:
model = load_model('model.h5')

with open('one_hot_encoder_geography.pkl', 'rb') as file:
    one_hot_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)



In [20]:
# example input
input_data = {
    'CreditScore': 600,
    'Gender': 'Male',
    'Age': 40,
    'Geography': 'France',
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts':2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary' : 50000
}

In [21]:
# Gender handling
input_data["Gender"] = label_encoder_gender.transform([input_data["Gender"]])[0] 

In [22]:
input_data

{'CreditScore': 600,
 'Gender': 1,
 'Age': 40,
 'Geography': 'France',
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000}

In [23]:
geo_constant = one_hot_encoder_geo.get_feature_names_out()
geo_constant

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [24]:
# geography handling
geo_values = one_hot_encoder_geo.transform([[input_data["Geography"]]]).toarray()
encoded_geo_df = pd.DataFrame(geo_values, columns= geo_constant) 
input_df = pd.concat([pd.DataFrame([input_data]).drop('Geography',axis=1), encoded_geo_df], axis=1)
input_df

d:\GEN-AI COURSE\PROJECTS\ANN CLASSIFICATION\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [ ]:
#Scale
input_df = scaler.transform(input_df)
input_df


array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [ ]:
# prediction
prediction = model.predict(input_df)
pred_prob= prediction[0][0]

if(pred_prob>0.5):
    print('Customer is likely to churn')
else:
    print('Customer is not likely to churn')

1/1 [==============================] - 2s 2s/step
Customer is not likely to churn
